# 0、前言

![Image Name](https://img2.baidu.com/it/u=1905334638,2514843252&fm=26&fmt=auto&gp=0.jpg)
在遥远的21世纪初，伴随着计算机硬件的发展和~~人民生活水平的提高~~
hinton等人提出的深度神经网络，机器学习，自然语言处理以及计算机视觉等领域都获得了长足的发展。
伴随着时间的进程
当初屠龙的少年，CNN不知不觉也已经成长为机器学习时代的巨龙。
近年来，CNN的发展进入了瓶颈期，长期对于卷积神经网络的依赖限制了计算机视觉领域的进一步发展。
于是，又出现了屠龙的少年，来自于异次元NLP
举起transformer的宝刀，一往无前！

犊子扯完，下面说正事，这是新开的对于transformer最新的论文讲解以及demo实验的系列。
作为该系列的第一篇，本文想要向大家介绍视觉transformer领域最常见的方案Vision Transformer（VIT）
以及一些基础知识。仅做分享，欢迎讨论。

作者：与贰 模式识别博士在读
版权声明：此文的所有文字、图片、代码以及相应数据文件的版权归本人(和鲸社区昵称：与贰)所有，文责自负。严禁任何商业性质的转载。公益性质转载需联系作者本人获取授权。转载本文时，请务必文字注明“来自：（和鲸社区：与贰）”，并附带本项目[超链接](https://www.heywhale.com/mw/project/60d980b694c44a0017dc0c5f?token=8ee278ebaa063c30)。

# 1、Motivation
## 1.1、Drawbacks in CNN
在计算机视觉领域，视觉信息通常可以被看为是像素组成的数组。这些数组通过卷积操作进行底层特征提取，语义信息分析并完成规定的任务比如图片分类，目标识别等等。
尽管基于CNN的各种网络已经在计算机视觉领域取得了巨大的进展，仍然有许多问题是需要被解决的。
### 1.1.1、关于像素的差异性
不同的任务通畅对于图片的像素会有不同的要求：在分类任务中，我们应当更多的关注于前景目标而忽略背景目标，在分割任务中，模型应该更多的考虑行人，而不是连成片的天空，道路，草地等等。然而，卷积网络却没有办法实现这个功能，他对于所有图像块/像素点均匀的进行处理而不考虑他们的重要性差异。这个缺陷会导致模型在空间特征的挖掘上比较低效，从而导致比较高的计算力以及比较差的特征表达能力。
### 1.1.2、关于图像的个性化
在使用卷积神经提取图像特征的时候，可以分为低级形状特征的提取以及高级语义信息的提取。这就导致了一个我们需要注意的问题，并不是所有的图片都会包含网络学习到的所有高级语义特征。低级的语义特征比如角落和边会在所有的自然图片中自然的存在，所以对于所有的图片使用一样的卷积核来提取低级语义特征是OK的。但是一些高级的语义特征，比如耳朵的形状，狗狗的形状，这些高级特征只会出现在特定的图片中而不是所有的图片，所以使用所有的高级语义特征提取的卷积核去对图片进行处理是高消耗而且不经济的。比如狗的一些特征是不会出现在只包含花，草，车辆的图片当中的，现有卷积网络的这种特性会导致大量的罕见卷积核被训练出来，并且大幅度增加网络的开销。
### 1.1.3、关于结构
每个卷积核都会对一小片区域进行处理，但是对于语义信息来说长依赖关系是十分重要的。为了增加这种长依赖关系，或者说空间-距离上的依赖关系，之前的方法尝试增大卷积核的大小，增加模型的深度，采用新的卷积方法比如空洞卷积等等。这些操作的本质都是希望增大感受野的面积，从而提高模型对于全局信息的关注能力。但是上述方法在某种程度上都增加了模型的复杂度和计算开销。

![Image Name](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fwww.pianshen.com%2Fimages%2F106%2Fa003bc575357859c67cb48634292a622.png&refer=http%3A%2F%2Fwww.pianshen.com&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1627472456&t=426386fb00142112465d0cc02a4003a5)



## 1.2、Why ViT?
为了解决这个问题，作者直接面向本质原因基于像素的卷积机制，提出了基于视觉的Transformer：vision transformer。那么为什么要提出Vision Transformer呢？我们尝试从自注意力机制和NLP中的transformer这两个角度来回答这个问题。并且在1.2.3小节中向大家介绍vit的主要内容。
### 1.2.1、自注意力机制
自注意力机制（self attention）是注意力机制中的一种，也是transformer中的重要组成部分。要想了解什么是自注意力机制，首先要给大家介绍什么是注意力机制。
#### 1.2.1.1 注意力机制
注意力机制的目的是根据特定的目标，去关注各个region和特定目标的关联程度。那么要完成这个任务，我们首先就要确定如何去计算这个关联程度。
拿两个句子来举例的话：
![Image Name](https://cdn.kesci.com/upload/image/qveu8hjzb0.png?imageView2/0/w/960/h/960)
要想知道这两句话表达的是不是同一个意思，我们首先要把每个单词去做word embedding，抽象成网络可以识别的向量。然后计算两个句子之间词与词的相似度，归一化后作为权重，并通过权重以及另一个句子的各个词向量，结合起来得到用另一个句子表示的该词的词向量。

假设我们对I分析，我们的query是I，我们计算I所对应的词向量和you，and，me这几个词向量之间的相似度，然后对相似度归一化得到权重再去和you，and，me几个词语相乘。这么操作我们就得到了第二句话根据I构建的词向量，然后不断用第二句话去表示第一句话的每个词，然后对比新句子和原句子之间的差异程度，就可以用来判断这两个橘子表达的是不是同一个意思。

#### 1.2.1.2 自注意力机制
上面所列举的例子，两句话表达的意思不一样，假设这两句话是同一句话，那么计算这个权重的过程就可以看为用自己表达自己，也就是在计算自己的每个lcoal之间的关联度。我们把这个过程称为self attention。这么做可以充分考虑不同的词向量之间的关联度，解决了之前提到的由于卷积带来的短依赖关系。

为了更好的去计算这个自注意力的过程，我们对于每个词向量都会抽象初query（Q），key(K)，value(V)这三个值来，然后根据下面的公示去计
![Image Name](https://cdn.kesci.com/upload/image/qveusecwbw.png?imageView2/0/w/640/h/640)
给定Target中的某个元素Query，通过计算Query和各个Key的相似性或者相关性，得到每个Key对应Value的权重系数，然后对Value进行加权求和，即得到了最终的Attention数值。所以本质上Attention机制是对Source中元素的Value值进行加权求和，而Query和Key用来计算对应Value的权重系数。

### 1.2.2、Transformer in NLP
Transformer在NLP中的大范围应用可以说起源于Attention is all you need这篇论文。
众所周知，这篇论文不仅开启了Transformer+大数据的新范式，
同时也开启了《XXXX is all you need》的灌水模式。
![Image Name](https://pic1.zhimg.com/80/v2-4b53b731a961ee467928619d14a5fd44_720w.jpg)

Encoder由N=6个相同的layer组成，layer指的就是上图左侧的单元，最左边有个“Nx”，这里是x6个。每个Layer由两个sub-layer组成，分别是multi-head self-attention mechanism和fully connected feed-forward network。其中每个sub-layer都加了residual connection和normalisation。

#### 1.2.2.1 Multi-head self-attention
就在上面我们刚刚给大家展示过self-attention的表示形式：
![Image Name](https://cdn.kesci.com/upload/image/qvev8r8aca.png?imageView2/0/w/640/h/640)

multi-head attention则是通过h个不同的线性变换对Q，K，V进行投影，最后将不同的attention结果拼接起来：
![Image Name](https://cdn.kesci.com/upload/image/qvev9v281a.png?imageView2/0/w/640/h/640)

#### 1.2.2.2 Position-wise feed-forward networks
这层主要是提供非线性变换。Attention输出的维度是\[batch_size*feature_dim,num_head*head_size\],第二个sub-layer是个全连接层，之所以是position-wise是因为过线性层时每个位置i的变换参数是一样的。

#### 1.2.2.3 Decoder
Decoder和Encoder的结构差不多，但是多了一个attention的sub-layer，这里先明确一下decoder的输入输出和解码过程：

输出：对应i位置的输出词的概率分布
输入：encoder的输出 & 对应i-1位置decoder的输出。所以中间的attention不是self-attention，它的K，V来自encoder，Q来自上一位置decoder的输出
解码：这里要注意一下，训练和预测是不一样的。在训练时，解码是一次全部decode出来，用上一步的ground truth来预测（mask矩阵也会改动，让解码时看不到未来的token）；而预测时，因为没有ground truth了，需要一个个预测。



### 1.2.3、VIT Backbone
为了将transformer的结构应用到视觉领域中来，VIT采取了将图片切块的思想，并将切块后的图片拉直，然后映射成类似word embedding的词向量。
在VIT 中学习和沿用了类似BERT的框架,在整个框架的第一个位置加入了一个可学习的cls-token 通过整个encoder 后得到分类用的特征,然后通过一个mlp 的header 得到最终的分类结果。
![Image Name](https://cdn.kesci.com/upload/image/qvevja4udf.png?imageView2/0/w/960/h/960)


# 2、代码讲解
这部分主要分成两个部分，分别介绍VIT中一些比较关键的代码，以及第二部分中基于VIT的代码提供了一个猫狗分类的Demo。

## 2.1、关键部分解读
VIT的结构其实就是取了完整transformer结构的一半——encoder部分来进行视觉信息的编码，所以VIT中需要讲解的关键部分包括，图像的分块和编码，位置编码以及分类token的作用这三个部分。
### 2.1.1、图像分块及编码
这一部分主要是通过将图像分为16*16的块，对于每个块将它相应的抽象为768维度的word embedding。
```
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
				## 根据实际图像的大小，来计算划分后patch的数量
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
				## 通过2D卷积来将切分后的patch块抽象成固定维度的特征表示
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x
```
### 2.1.2、位置编码和分类token
位置编码主要是用来表达各个词向量之间的相对位置关系，在transformer原文中提出使用正余弦的方式表达，而在VIT中作者表示可以直接设置可学习的参数形式在训练过程中进行学习。相关的代码表示一行可见。
```
self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
```
分类token就是模仿NLP，加入了一个额外的词向量，这个向量伴随着网络的训练，学习全局信息的编码，并且在最后被送入分类层进行分类使用。相关的代码和位置编码部分类似，也是通过一行代码就可以表达。
```
self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
```

## 2.2、完整Demo实现
在这个部分给大家伙儿展示一个基于VIT的猫狗二分类代码演示，至于为什么是猫狗二分类，是因为简单啊。上次写的时候随手找了个猫狗的数据集跑通了。既然这个数据集可以简洁明了的给大家展示怎么快速搭建一个demo，我认为这就足够了。
### 2.2.1、数据准备及各种各样包的加载

In [1]:
# 各种包的加载
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms, models, utils
from torch import nn, einsum
import torch.nn.functional as F
from tqdm.notebook import tqdm
# from tqdm import tqdm_notebook as tqdm
from torch import nn
import matplotlib.pyplot as plt
import torch.optim as optim
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [2]:
# 数据的加载和切分工作
class MyDataset(Dataset):

    def __init__(self, data_path: str, train=True, transform=None):
        self.data_path = data_path
        self.train_flag = train
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),  #尺寸规范
                transforms.ToTensor(),  #转化为tensor
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        else:
            self.transform = transform
        self.path_list = os.listdir(data_path)

    def __getitem__(self, idx: int):
        # img to tensor and label to tensor
        img_path = self.path_list[idx]
        if self.train_flag is True:
            if img_path.split('.')[0] == 'dog':
                label = 1
            else:
                label = 0
        else:
            label = int(img_path.split('.')[0])  # split 的是str类型要转换为int
        label = torch.as_tensor(
            label, dtype=torch.int64)  # 必须使用long 类型数据，否则后面训练会报错 expect long
        img_path = os.path.join(self.data_path, img_path)
        img = Image.open(img_path)
        img = self.transform(img)
        return img, label

    def __len__(self) -> int:
        return len(self.path_list)


# !ls "/home/mw/input/cat_dog1615/data/"
train_dir = "C:/Users/Harri/my_jupyter_notebook/data_cat-dog/train/"
train_ds = MyDataset(train_dir)
test_dir = "C:/Users/Harri/my_jupyter_notebook/data_cat-dog/test/"
test_ds = MyDataset(train_dir)
full_ds = train_ds
train_size = int(0.8 * len(full_ds))
validate_size = len(full_ds) - train_size
new_train_ds, validate_ds = torch.utils.data.random_split(
    full_ds, [train_size, validate_size])
# 调用torch的Dataloader来对数据进行封装成可以使用的pipeline
new_train_loader = torch.utils.data.DataLoader(new_train_ds,
                                               batch_size=32,
                                               shuffle=True,
                                               pin_memory=True,
                                               num_workers=0)
validate_loader = torch.utils.data.DataLoader(validate_ds,
                                              batch_size=32,
                                              shuffle=True,
                                              pin_memory=True,
                                              num_workers=0)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'C:/Users/Harri/my_jupyter_notebook/data_cat-dog/train/'

### 2.2.2、backbone的关键代码
这里展示了一个VIT的backbone以及它依赖的各种函数

In [3]:
# 判断t是不是一个tuple形式，如果不是就把它变成tuple形式。
def pair(t):
    return t if isinstance(t, tuple) else (t, t)


### 在res链接之前的残差模块
class PreNorm(nn.Module):

    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)


###包含了全连接层，Gelu以及dropout的前向网络
class FeedForward(nn.Module):

    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim, hidden_dim), nn.GELU(),
                                 nn.Dropout(dropout),
                                 nn.Linear(hidden_dim,
                                           dim), nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)


###multi-head attention模块
class Attention(nn.Module):

    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head**-0.5

        self.attend = nn.Softmax(dim=-1)
        ##初始化qkv的为可学习的全联接层
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)) if project_out else nn.Identity()

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=h), qkv)

        dots = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale

        attn = self.attend(dots)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


###完整的transformer模块
class Transformer(nn.Module):

    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(
                nn.ModuleList([
                    PreNorm(
                        dim,
                        Attention(dim,
                                  heads=heads,
                                  dim_head=dim_head,
                                  dropout=dropout)),
                    PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
                ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x


class ViT(nn.Module):

    def __init__(self,
                 *,
                 image_size,
                 patch_size,
                 num_classes,
                 dim,
                 depth,
                 heads,
                 mlp_dim,
                 pool='cls',
                 channels=3,
                 dim_head=64,
                 dropout=0.,
                 emb_dropout=0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        ##对比原图是不是完整可分
        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width //
                                                        patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {
            'cls', 'mean'
        }, 'pool type must be either cls (cls token) or mean (mean pooling)'

        ##将patch embedding转化为固定长度的词向量编码
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)',
                      p1=patch_height,
                      p2=patch_width),
            nn.Linear(patch_dim, dim),
        )
        ##位置编码
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        ## 分类token
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim,
                                       dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(nn.LayerNorm(dim),
                                      nn.Linear(dim, num_classes))

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim=1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

### 2.2.3、训练过程


In [4]:
#定义一些工具类
class AvgrageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt


def accuracy(output, label, topk=(1, )):
    maxk = max(topk)
    batch_size = label.size(0)

    # 获取前K的索引
    _, pred = output.topk(maxk, 1, True, True)  #使用topk来获得前k个的索引
    pred = pred.t()  # 进行转置
    # eq按照对应元素进行比较 view(1,-1) 自动转换到行为1,的形状， expand_as(pred) 扩展到pred的shape
    # expand_as 执行按行复制来扩展，要保证列相等
    correct = pred.eq(label.view(
        1, -1).expand_as(pred))  # 与正确标签序列形成的矩阵相比，生成True/False矩阵
    #     print(correct)

    rtn = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(
            0)  # 前k行的数据 然后平整到1维度，来计算true的总个数
        rtn.append(correct_k.mul_(
            100.0 / batch_size))  # mul_() ternsor 的乘法  正确的数目/总的数目 乘以100 变成百分比
    return rtn

In [7]:
# 网络的训练代码
def train(epoch, train_loader, device, model, criterion, optimizer):
    model = model.to(device)
    for e in range(epoch):
        model.train()
        top1 = AvgrageMeter()
        train_loss = 0.0
        train_loader = tqdm(train_loader)
        train_loader.set_description('[%s%04d/%04d %s%f]' %
                                     ('Epoch:', e + 1, epoch, 'lr:', 0.001))
        for i, data in enumerate(train_loader, 0):  # 0是下标起始位置默认为0
            inputs, labels = data[0].to(device), data[1].to(device)
            # 初始为0，清除上个batch的梯度信息
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # topk 准确率计算
            prec1, prec2 = accuracy(outputs, labels, topk=(1, 2))
            n = inputs.size(0)
            top1.update(prec1.item(), n)
            train_loss += loss.item()
            postfix = {
                'train_loss': '%.6f' % (train_loss / (i + 1)),
                'train_acc': '%.6f' % top1.avg
            }
            train_loader.set_postfix(log=postfix)
            ######不想等他训练完，所以我break了，实际使用请注释掉
            break
    print('Finished Training')


net = ViT(image_size=224,
          patch_size=32,
          num_classes=2,
          dim=1024,
          depth=6,
          heads=16,
          mlp_dim=2048,
          dropout=0.1,
          emb_dropout=0.1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
train(1, new_train_loader, device, net, criterion, optimizer)

  0%|          | 0/500 [00:00<?, ?it/s]

Finished Training


# 3、Ending
## 3.1、总结及展望
ViT（vision transformer）是Google在2020年提出的直接将transformer应用在图像分类的模型，后面很多的工作都是基于ViT进行改进的。
ViT算是一个很好的开始，虽然也存在一些问题，但是至少证明了纯粹的transformer在CV领域应用的可能性。
VIT中一个比较严重的问题就是需要超大的数据集去进行预训练
这最起码证明了一个问题，Money is all you need！
## 3.2、碎碎念的结束语
各位读者老爷大家好，本期内容到这里就结束啦！
这里为了简单（偷懒）仅仅是训练了一个epoch保证代码的通用性。（轻喷）
这是Transformer小试牛刀系列第一篇，欢迎大家来和鲸平台一起参与讨论啦！
相关的代码均可开源自用，商业用途除外，各位读者老爷请自取。
目前代码支持cpu和gpu下训练。
挥手🙋‍♂️，我们下次再见